In [2]:
import json
import os
import nltk
import tqdm
from nltk.stem.snowball import EnglishStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
os.chdir("C:/Users/ADMIN/Desktop/mindicator/")

# data_path =     '../res/data/mIndicator_dataset/raw_data/train'
# key_data_path = '../res/data/mIndicator_dataset/raw_data/class_definitions'
out_path =      'C:/Users/ADMIN/Desktop/mindicator/data/mIndicator_dataset/labelled_data/labelled_train_data.json'


In [5]:
stemmer = EnglishStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

#part-of-speech (POS) tagger based on Maximum Entropy (ME) principles written for NLTK.
tagger = nltk.data.load("taggers/maxent_treebank_pos_tagger/english.pickle")
#wh-pronoun,wh-abverb,wh-determiner,possessive wh-pronoun
Wh_tags = ['WP','WRB','WDT','WP$']

labeled_data = {}
key_data = {}

#finding question that where asked in the chats 
def isQuestion(sent):
    sent = sent.lower()
    tokens = nltk.word_tokenize(sent)
    tags = tagger.tag(tokens)

    if '?' in sent:
    return True

    for tag in tags:
    if tag[1] in Wh_tags:
            return True
      
    if (len(tokens)>=2 and tokens[-2]=='or' and tokens[-1]=='not') or (len(tokens)>=3 and tokens[-3]=='or' and tokens[-2]=='not'):
        return True
    
    #vbz=verb, [rb ,rbr , rbs ] =adverb, dt=determiner ,ex = existential there  
    if (len(tags)>=3 and tags[0][1] in ['VBZ'] and tags[1][1] in ['RB','RBR','RBS'] and tags[2][1] in ['DT','EX']) or\
        (len(tags)>=2 and tags[0][1] in ['VBZ'] and tags[1][1] in ['DT','EX']):
        return True

    return False



In [18]:
key_data_path = 'C:/Users/ADMIN/Desktop/mindicator/data/mIndicator_dataset/raw_data/class_definitions'
#navigating to class definaiton of train : accident,dely cancel  
for file in os.listdir(key_data_path):
    file_path = os.path.join(key_data_path,file)
    
    with open(file_path, 'r', encoding="utf8", errors='ignore') as f:   #loading the files 
        lines = f.readlines()     # corpus of delay of trains 

    _class = '.'.join(file.split('.')[:-1]) # delay of trains  file
    key_data[_class] = []
    for line in lines:
        stemmed = " ".join([stemmer.stem(w) for w in word_tokenize(line)])
        key_data[_class].append(stemmed)
        
        
        

        

In [8]:
data_path = 'C:/Users/ADMIN/Desktop/mindicator/data/mIndicator_dataset/raw_data/train'

# making our loops  for showing  a smart progress meter - just wrap any iterable by using tqdm
for file in tqdm.tqdm(os.listdir(data_path)):
    file_path = os.path.join(data_path,file)

    with open(file_path, 'r', encoding="utf8", errors='ignore') as f:
        print("Reading file - " + file_path + " ...")
        try:
            data_json = json.load(f)
        except Exception as e:
            print("Exception while reading file - " + file_path + " !")
            print(str(e))
            data_json = {}

    messages_arr_stemmed = []

    if bool(data_json):
        for timestamp, data in data_json.items():
            msg_lemmed = " ".join([wordnet_lemmatizer.lemmatize(w) for w in word_tokenize(data['m'])]) #lemmatizing
            msg_stemmed = " ".join([stemmer.stem(msg) for msg in word_tokenize(msg_lemmed)])           #stemming 
            messages_arr_stemmed.append([msg_stemmed,timestamp])
         
        #finding questions  for the messages 
        for stemmed_msg, timestamp in messages_arr_stemmed:
            if isQuestion(data_json[timestamp]['m']):
                if labeled_data.get('questions') is None:
                    labeled_data['questions'] = []
                labeled_data['questions'].append(data_json[timestamp])
            else:
                for _class, _ in key_data.items():
                    for keyword in key_data[_class]:
                        if keyword in stemmed_msg:
                            if labeled_data.get(_class) is None:
                                labeled_data[_class] = []
                            labeled_data[_class].append(data_json[timestamp])


                            
                                           

with open(out_path, 'w', encoding="utf8", errors='ignore') as f:
    print("Dumping output to JSON file...")
    json.dump(labeled_data, f)

  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

Reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180701_CENTRAL_LINE.txt ...


  3%|██▋                                                                                | 1/31 [00:22<11:27, 22.90s/it]

Reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180702_CENTRAL_LINE.txt ...


  6%|█████▎                                                                             | 2/31 [02:56<29:57, 61.98s/it]

...

Reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180725_CENTRAL_LINE.txt ...
Exception while reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180725_CENTRAL_LINE.txt !
Expecting value: line 1 column 1 (char 0)
Reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180726_CENTRAL_LINE.txt ...
Exception while reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180726_CENTRAL_LINE.txt !
Expecting value: line 1 column 1 (char 0)
Reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180727_CENTRAL_LINE.txt ...
Exception while reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180727_CENTRAL_LINE.txt !
Expecting value: line 1 column 1 (

 90%|██████████████████████████████████████████████████████████████████████████        | 28/31 [13:58<00:18,  6.16s/it]

Reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180729_CENTRAL_LINE.txt ...
Exception while reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180729_CENTRAL_LINE.txt !
Expecting value: line 1 column 1 (char 0)
Reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180730_CENTRAL_LINE.txt ...
Exception while reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180730_CENTRAL_LINE.txt !
Expecting value: line 1 column 1 (char 0)
Reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180731_CENTRAL_LINE.txt ...
Exception while reading file - C:/Users/ADMIN/Desktop/Train-Status-Notifier-master/res/data/mIndicator_dataset/raw_data/train\20180731_CENTRAL_LINE.txt !
Expecting value: line 1 column 1 (

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [13:58<00:00, 27.06s/it]


Dumping output to JSON file...


In [35]:
labeled_data["questions"]   #prints all thedetails of the  messages in which questions are asked 

[{'i': '106324083053110511194',
  'lc': 0,
  'm': 'Hi, Kasara to CST train is running?',
  'n': 'Sandip Wakchoure',
  'ri': '',
  'rm': '',
  'rn': '',
  'sc': 0,
  't': '02 Jul 05:20 AM',
  'time': 1530489001356},
 {'i': '101992906466069519545',
  'lc': 0,
  'm': 'where is khopoli csmt fast (sent from Dombivli Stn.)',
  'n': 'Shreyas Patwardhan',
  'ri': '',
  'rm': '',
  'rn': '',
  'sc': 0,
  't': '02 Jul 06:15 AM',
  'time': 1530492320317},
 {'i': '105055236475602144795',
  'lc': 0,
  'm': 'hi.. is 6.50 train from thane to csmt from.platform 3, a regular one ? (sent from Thane Stn.)',
  'n': 'Nihu prabhudesai',
  'ri': '',
  'rm': '',
  'rn': '',
  'sc': 0,
  't': '02 Jul 06:35 AM',
  'time': 1530493517152},
 {'i': '111735761878728446036',
  'lc': 0,
  'm': 'status of 7:04 fast local for csmt? (sent from Kalva)',
  'n': 'Jeetesh Rai',
  'ri': '',
  'rm': '',
  'rn': '',
  'sc': 0,
  't': '02 Jul 07:10 AM',
  'time': 1530495635274},
 {'i': '109184214127388685770',
  'lc': 0,
  'm': 

In [9]:
import os
import json
import nltk
import random
import re

#corpus for fnding sentiment of message 
classes_under_consideration = ['ynQuestion','whQuestion','Greet','Statement','Emotion']

out_dir = './../res/data/nps_chat_dataset'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

#corpus reader
posts = nltk.corpus.nps_chat.xml_posts()[:]

dataset = {} 
for post in posts:
    _class = post.get('class')
    if _class not in classes_under_consideration:
        continue
    text = " "
    for word in nltk.word_tokenize(post.text):
        if not re.search('user', word, re.IGNORECASE):
            text = text + " " + word.lower()
    text = text.strip()
    if dataset.get(_class) == None:
        dataset[_class] = []
    if _class not in ['ynQuestion','whQuestion'] and len(text) > 3:
        dataset[_class].append(text)
    elif _class in ['ynQuestion','whQuestion']:
        dataset[_class].append(text)


for _class, texts in dataset.items():
    texts = random.sample(texts,533)       #questions asked in message
    file_name = '{}.txt'.format(_class)
    with open(os.path.join(out_dir,file_name), 'w') as f:
        f.write('\n'.join(texts))                      #store in question file at specified path 

In [51]:
texts

['you are ? ?',
 'any ladies care to chat',
 'yeah ... life , you know ?',
 'are you a male ?',
 'history ?',
 'do we really need to go there..lol',
 'hello room anyone wan na chat with 23 female from virginia',
 'you work in the bear place in maidstone opposite the coffe placce ?',
 '... high beams ?',
 'ohio anybody ? ? ?',
 'question number 1 : do you like to play with dolls',
 'ya need some root b gone or something ?',
 'can i do a quiz',
 'drop something on your foot ?',
 "the teacher would probably be 80 would n't she ?",
 'yeah baby ? ? ?',
 'you mean the price of homes ?',
 'did give you those ?',
 'any one want to chat',
 'any girls want to talk dirty',
 'or too smart ?',
 'does that make you a redneck ?',
 'sucks being you then huh ? j/k : )',
 'any guys wan na chat with a very nauty girl',
 'were attracted or are attracted',
 'any austin texas area ladies in here',
 'any girls wan na chat with 24/m',
 'awww yer gon na be what ? 57 right ?',
 'a beach ball ? ?',
 'wow ... are

In [26]:
# from wordcloud import WordCloud, STOPWORDS 
# import matplotlib.pyplot as plt 
# import pandas as pd 
  
# wordcloud = WordCloud(width = 800, height = 800, 
#                 background_color ='white', 
#                 stopwords = stopwords, 
#                 min_font_size = 10).generate(labeled_data) 
  
# # plot the WordCloud image                        
# plt.figure(figsize = (8, 8), facecolor = None) 
# plt.imshow(wordcloud) 
# plt.axis("off") 
# plt.tight_layout(pad = 0) 
  
# plt.show() 

# Model Training (classifying the dialogue type)

In [45]:
import nltk
#corpus reader
posts = nltk.corpus.nps_chat.xml_posts()[:]

def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

feature_making  = [(dialogue_act_features(post.text), post.get('class')) for post in posts]

size = int(len(feature_making ) * 0.1)

train_set, test_set = feature_making[size:], feature_making [:size]

#uisng naive bayes classifier algorithm
nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

#testing
print("score : ",nltk.classify.accuracy(nb_classifier, test_set))

score :  0.6685606060606061
